In [18]:
import graph_tool.all as gt
from tqdm.notebook import tqdm
import ast
import pandas as pd
import requests

# графы связей реальных людей

In [129]:
real_people = pd.read_csv('real_people.csv')

In [130]:
real_people = real_people.drop(columns=['Unnamed: 0'])

In [131]:
d = [tuple(x) for x in real_people.to_records(index=False)]

list_normal = []
for ids in tqdm(d):
    list_normal.append(ids[0])
    list_normal.append(ids[1])
list_normal = list(set(list_normal))

dick = {}
for ids in tqdm(enumerate(list_normal)):
    dick[ids[1]] = ids[0]
    
new_d = []
for i1, i2 in tqdm(d):
    new_d.append((dick[i1], dick[i2]))

In [ ]:
seen_list = [7586]
subscriptions_dict = {}
subscriptions_count_dict = {}
post_dict = {}
post_count_dict = {}
user_information = {}
user_info = {}
TOKEN = 'ef53df19fa9a09240d437cf80bccdae854dec9c40211165ec112d6b5928690ff409ac4eaf44be73a7c7cd'
data = requests.get(
    'https://api.vk.com/method/friends.get', 
    params={
        "user_id": 7586,
        "v":"5.92",
        "count": 600,
        "access_token": TOKEN
    }
    ).json()


for user_id in tqdm(data['response']['items']):
    seen_list.append(user_id)

In [141]:
new_seen = [dick[s] for s in seen_list]

In [142]:
%%time
g = gt.Graph(directed=False)
g.add_edge_list(new_d)

CPU times: user 54.1 ms, sys: 103 ms, total: 157 ms
Wall time: 232 ms


In [ ]:
def get_graph_feat(new_seen):
    s_dists = []
    for s in tqdm(new_seen):
        dist = gt.shortest_distance(g, source=s)
        s_dists.append(dist)
    s_dists1 = [list(sd) for sd in tqdm(s_dists)]
    max_dists = [max(x) for x in tqdm(s_dists1)]
    mean_dists = [sum(x)/len(x) for x in tqdm(s_dists1)]
    degrees = g.get_out_degrees(new_seen)
    closenesses = []
    for ns in tqdm(new_seen):
        closenesses.append(gt.closeness(g, source=ns))
    closenesses_real =[float(x) for x in tqdm(closenesses)]
    return max_dists, mean_dists, degrees, closenesses_real

In [ ]:
max_dists, mean_dists, degrees, closenesses_real = get_graph_feat(new_seen)

In [ ]:
graph_real = pd.DataFrame({'id': new_seen, 'max_dists':max_dists, 'mean_dists':mean_dists, 'degrees':degrees, 'closenesses_real':closenesses_real})

In [ ]:
def get_key(value):
    for k, v in dick.items():
        if v == value:
            return k

In [ ]:
graph_real['id_vk'] = graph_real.id.apply(get_key)

In [ ]:
graph_real.to_csv('graph_metrics_real.csv')

# графы связей б

In [120]:
bots = pd.read_csv('graph_metrics_bot 2.csv')

In [121]:
bots = bots.drop('Unnamed: 0', axis=1)

In [122]:
d = [(int(x[0]), int(x[1])) for x in bots.to_records(index=False)]

list_bots = []
for ids in tqdm(d):
    list_bots.append(int(ids[0]))
    list_bots.append(int(ids[1]))
list_bots = list(set(list_bots))

dick = {}
for ids in tqdm(enumerate(list_bots)):
    dick[ids[1]] = ids[0]

new_d = []
for i1, i2 in tqdm(d):
    new_d.append((dick[i1], dick[i2]))

  0%|          | 0/9136846 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/9136846 [00:00<?, ?it/s]

In [118]:
seen_list = [656481023]
subscriptions_dict = {}
subscriptions_count_dict = {}
post_dict = {}
post_count_dict = {}
user_information = {}
user_info = {}
TOKEN = 'ef53df19fa9a09240d437cf80bccdae854dec9c40211165ec112d6b5928690ff409ac4eaf44be73a7c7cd'
data = requests.get('https://api.vk.com/method/users.getFollowers', 
    params={
        "user_id": 656481023,
        "v":"5.92",
        "count": 807,
        "access_token": TOKEN
    }
    ).json()


for user_id in tqdm(data['response']['items']):
    seen_list.append(user_id)

  0%|          | 0/807 [00:00<?, ?it/s]

In [134]:
new_seen = []
old_guys = 0
new_guys = 0
for s in tqdm(seen_list):
    if s in list_bots:
        old_guys += 1
        new_seen.append(dick[s])
    else:
        new_guys += 1

  0%|          | 0/9402204 [00:00<?, ?it/s]

In [125]:
%%time
g = gt.Graph(directed=False)
g.add_edge_list(new_d)

CPU times: user 5.17 s, sys: 464 ms, total: 5.64 s
Wall time: 6.02 s


In [ ]:
max_dists, mean_dists, degrees, closenesses_real = get_graph_feat(new_seen)

In [ ]:
graph_bot = pd.DataFrame({'id': new_seen, 'max_dists':max_dists, 'mean_dists':mean_dists, 'degrees':degrees, 'closenesses_real':closenesses_real})
graph_bot['id_vk'] = graph_bot.id.apply(get_key)
graph_bot.to_csv('graph_metrics_bot.csv')

In [ ]:
real_people = pd.read_csv('graph_metrics_real.csv')
real_people['target'] = 1
bots = pd.read_csv('graph_metrics_bot.csv')
bots['target'] = 0

In [ ]:
all_graph_metrics = pd.concat([real_people, bots], ignore_index=True)
all_graph_metrics = all_graph_metrics.drop('Unnamed: 0', axis=1)
all_graph_metrics.to_csv('graph_metrics_all.csv')